In [ ]:
import json
import sqlite3
import time
from collections import Counter
from datetime import datetime

import numpy as np
from scipy.stats import chi2_contingency
from tabulate import tabulate

In [ ]:
books_file = '/Users/andre/Downloads/archive/lighter_books.json'
authors_file = '/Users/andre/Downloads/archive/lighter_authors.json'
list_file = '/Users/andre/Downloads/archive/list.json/list.json'

conn = sqlite3.connect('tables.db')
cursor = conn.cursor()

cursor.execute("""
                    CREATE TABLE IF NOT EXISTS authors
                    (
                        id INT,
                        ratings_count INT,
                        average_rating FLOAT,
                        text_reviews_count INT,
                        name TEXT,
                        gender TEXT,
                        about TEXT,
                        fans_count INT
                    )
                """)

cursor.execute("""
                    CREATE TABLE IF NOT EXISTS book_to_author
                    (
                        author_id INT,
                        book_id INT
                    )
                """)

cursor.execute("""
                    CREATE TABLE IF NOT EXISTS books
                    (
                        id INT,
                        title TEXT,
                        author_id INT,
                        language TEXT,
                        average_rating FLOAT,
                        rating_dist_1 INT,
                        rating_dist_2 INT,
                        rating_dist_3 INT,
                        rating_dist_4 INT,
                        rating_dist_5 INT,
                        ratings_count INT,
                        text_reviews_count INT,
                        publication_date TEXT,
                        original_publication_date TEXT,
                        format TEXT,
                        num_pages INT,
                        series_id INT,
                        description TEXT
                    )
                """)

cursor.execute("""
                    CREATE TABLE IF NOT EXISTS worst_books_list
                    (
                        book_id INT,
                        title TEXT,
                        author_id INT,
                        author TEXT
                    )
                """)

In [ ]:
with open(books_file, 'r') as f:

    for line in f:
        
        if len(line) > 1:

            entry = json.loads(line)

            book_id = int(entry.get('id'))
            title = str(entry.get('title'))
            author_id = int(entry.get('author_id'))
            language = str(entry.get('language'))
            average_rating = float(entry.get('average_rating'))
            rating_dist = [int(rating[2:]) for rating in entry.get('rating_dist', '').split('|')[:-1]]
            rating_dist_1 = rating_dist[4]
            rating_dist_2 = rating_dist[3]
            rating_dist_3 = rating_dist[2]
            rating_dist_4 = rating_dist[1]
            rating_dist_5 = rating_dist[0]
            ratings_count = int(entry.get('ratings_count'))
            text_reviews_count = int(entry.get('text_reviews_count'))
            publication_date = str(entry.get('publication_date'))
            if len(publication_date) == 4:
                publication_date = publication_date + '-01-01'
            elif len(publication_date) == 7:
                publication_date = publication_date + '-01'
            elif len(publication_date) == 10:
                publication_date = publication_date
            else:
                publication_date = None
            original_publication_date = str(entry.get('original_publication_date'))
            if len(original_publication_date) == 4:
                original_publication_date = original_publication_date + '-01-01'
            elif len(original_publication_date) == 7:
                original_publication_date = original_publication_date + '-01'
            elif len(original_publication_date) == 10:
                original_publication_date = original_publication_date
            else:
                original_publication_date = None
            format = str(entry.get('format'))
            try:
                num_pages = int(entry.get('num_pages'))
            except:
                num_pages = None
            try:
                series_id = int(entry.get('series_id'))
            except:
                series_id = None
            description = str(entry.get('description'))

            cursor.execute("""
                                INSERT INTO books
                                (
                                    id,
                                    title,
                                    author_id,
                                    language,
                                    average_rating,
                                    rating_dist_1,
                                    rating_dist_2,
                                    rating_dist_3,
                                    rating_dist_4,
                                    rating_dist_5,
                                    ratings_count,
                                    text_reviews_count,
                                    publication_date,
                                    original_publication_date,
                                    format,
                                    num_pages,
                                    series_id,
                                    description
                                )
                                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                            """,
                           (book_id,
                            title,
                            author_id,
                            language,
                            average_rating,
                            rating_dist_1,
                            rating_dist_2,
                            rating_dist_3,
                            rating_dist_4,
                            rating_dist_5,
                            ratings_count,
                            text_reviews_count,
                            publication_date,
                            original_publication_date,
                            format,
                            num_pages,
                            series_id,
                            description))

conn.commit()
cursor.close()
conn.close()

In [ ]:
with open(authors_file, 'r') as f:

    for line in f:
        
        if len(line) > 1:
    
            entry = json.loads(line)

            author_id = int(entry.get('id'))
            book_ids = entry.get('book_ids')

            for book_id in book_ids:

                cursor.execute("""
                                    INSERT INTO book_to_author
                                    (
                                        author_id,
                                        book_id
                                    )
                                    VALUES (?, ?)
                                """,
                               (author_id,
                                int(book_id)))
                
conn.commit()
cursor.close()
conn.close()

In [ ]:
with open(authors_file, 'r') as f:

    for line in f:
        
        if len(line) > 1:
    
            entry = json.loads(line)

            ratings_count = int(entry.get('ratings_count'))
            average_rating = float(entry.get('average_rating'))
            text_reviews_count = int(entry.get('text_reviews_count'))
            id = int(entry.get('id'))
            name = str(entry.get('name'))
            gender = str(entry.get('gender'))
            about = str(entry.get('about'))
            fans_count = int(entry.get('fans_count'))

            cursor.execute("""
                                INSERT INTO authors 
                                (
                                    id,
                                    ratings_count,
                                    average_rating,
                                    text_reviews_count,
                                    name,
                                    gender,
                                    about,
                                    fans_count
                                )
                                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                            """,
                           (id,
                            ratings_count,
                            average_rating,
                            text_reviews_count,
                            name,
                            gender,
                            about,
                            fans_count))

conn.commit()
cursor.close()
conn.close()

In [ ]:
with open(list_file, 'r') as f:
    for line in f:
        item = json.loads(line)
        if item['title'] == 'The Worst Books of All Time':
            for book in item['books']:

                book_id = int(book.get('book_id'))
                title = str(book.get('title'))
                author_id = int(book.get('author_id'))
                author = str(book.get('author'))

                cursor.execute("""
                                        INSERT INTO worst_books_list
                                        (
                                            book_id,
                                            title,
                                            author_id,
                                            author
                                        )
                                        VALUES (?, ?, ?, ?)
                                    """,
                                   (book_id,
                                                title,
                                                author_id,
                                                author))

conn.commit()
cursor.close()
conn.close()

In [ ]:
cursor.execute("create index book_index on books(id)")
conn.commit()
cursor.execute("create index author_index on authors(id)")
conn.commit()
cursor.execute("create index book_to_author_index on book_to_author(book_id)")
conn.commit()

cursor.close()
conn.close()

In [ ]:
conn = sqlite3.connect('/Users/andre/PycharmProjects/ADM-HW2/tables.db')
cursor = conn.cursor()

# Probability = (Number of books with > 30% 5-star ratings) / (Total number of books)
# (235633, 7027431)
query1 = """
            select sum(case when rating_dist_5 / ratings_count > 0.3 then 1 else 0 end) as count_over_30,
                   count(id) as total_count
            from books
         """

# Probability = (Number of authors with new book within two years from their last work) / (Total number of authors)
# 120748
query2 = """
            with tmp as
            (
            select  a.author_id,
                    a.original_publication_date,
                    date(a.original_publication_date) as original_publication_date,
                    row_number() over(partition by a.author_id order by date(a.original_publication_date) desc) rank
            from books a
            inner join book_to_author b
                on a.id = b.book_id
            )
            select sum(case when (strftime('%s', a.original_publication_date) - strftime('%s', b.original_publication_date)) / 86400 <= 730 then 1 else 0 end) as result
            from (select author_id, original_publication_date from tmp where rank = 1) a
            inner join (select author_id, original_publication_date from tmp where rank = 2) b
                on a.author_id = b.author_id
         """

# 351767
query3 = """
            select count(*)
            from authors
         """

# Probability = (Number of worst books with more than 700 pages) / (Total number of books with more than 700 pages)
# 220
query4 = """
            select count(*)
            from worst_books_list a
            inner join books b
                on a.book_id = b.id    
            where b.num_pages > 700
         """

# 111120
query5 = """
            select count(*)
            from books
            where num_pages > 700
         """

# Probability = Total number of worst books / Total number of books
# 7393
# 7393 / 7027431 = 0,00105202
query6 = """
            select count(*)
            from worst_books_list
         """

cursor.execute(query1)
output = cursor.fetchall()
print(output)

cursor.close()
conn.close()

In [ ]:
# Probability = Total number of books with more than 700 pages / Total number of books
# 111120 / 7027431 = 0,01581232

# Probability = Number of worst books with more than 700 pages / Total number of books
# 220 / 7027431 = 0,00003131

# Joint probability
# 0,00105202 * 0,01581232 = 0,00001663

#                                           Y = Yes (Over 700 pages)	Y = No (Not Over 700 pages)
#    X = Yes (Inside "Worst Books")	                    A	                        B
#    X = No (Not Inside "Worst Books")	                C	                        D

# ([A, B], [C, D])
observed_values = np.array([[220, 5517], [110900, 3838031]])
chi2, p, dof, expected = chi2_contingency(observed_values)

if p < 0.05:
    print("Reject the null hypothesis: X and Y are not independent.")
else:
    print("Fail to reject the null hypothesis: X and Y are independent.")